In [44]:
#export
from fastai.text import *

In [45]:
#export
def return_path_dict(name, url):
    path = {}
    path[name] = untar_data(url)
    print(path[name])
    print(path[name].ls())
    return path

In [3]:
path = {}
path['imdb'] = untar_data(URLs.IMDB_SAMPLE)
print()

In [14]:
path = return_path_dict('imdb', URLs.IMDB_SAMPLE)

/home/ravivijay/.fastai/data/imdb_sample
[PosixPath('/home/ravivijay/.fastai/data/imdb_sample/texts.csv')]


In [19]:
pd.read_csv(path['imdb']/'texts.csv')[:5]

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


### Language Model

In [24]:
data= {}
data['lm'] = (TextList.from_csv(path['imdb'], 'texts.csv', cols ='text')
              .split_by_rand_pct()
              .label_for_lm()
              .databunch()
             )
data['lm'].save()

In [25]:
data['lm'].show_batch()

idx,text
0,"! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk ! xxbos xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is"
1,xxmaj igor xxmaj xxunk was horribly burned and killed in a lab accident . \n \n xxmaj then a woman xxmaj xxunk ( xxmaj xxunk xxmaj xxunk ) that xxmaj igor hired before his death to xxunk him in his xxunk shows up . xxmaj yanos agrees to hire her in place of his brother and together they seek the xxunk for the xxunk of dead xxunk . xxmaj
2,"of one big mine , it is divided at the border and xxmaj german workers are not welcome in the xxmaj french mine , despite there being greater xxunk in xxmaj germany . xxmaj this , language differences ( illustrated wonderfully in a dance hall scene ) and xxup xxunk xxunk to create a huge xxunk between the xxunk -- resulting in a xxup we vs. xxup they mentality ."
3,"'s why most other people who have seen it , saw it . xxmaj but if you 're not on the xxunk for xxmaj xxunk ; there really is no reason to bother with this one . xxbos i managed to grab a viewing of this with the aid of xxup mst3 k , and oh boy , even with the xxunk this movie was excruciatingly bad . xxmaj imagine"
4,"spent his work on the storyboard . xxmaj alongside this , the xxunk spent far too much time leaning angles that do not match the message of the movie . xxmaj the funniest is , however , if you take a look at the movie 's website , you can read that it was on purpose that the director has chosen to make the film with bad camera angles ."


In [29]:
learn = language_model_learner(data['lm'], AWD_LSTM)
learn.fit_one_cycle(2,1e-2)
learn.save('langmodel-imdb-qs-1')
learn.save_encoder('imdb-qs-encoder-1')

epoch,train_loss,valid_loss,accuracy,time
0,4.333920,3.754129,0.292560,00:03
1,4.062951,3.721723,0.294658,00:03


In [31]:
learn.show_results()

text,target,pred
would think that there would have been a little more imagination and xxunk to even a film like this .,i just kept waiting for something to happen of any importance as people stood around making xxunk and acting like,xxmaj think thought watching for the to be . the kind . the were up for a . xxunk .
"the room briefly , and comes back to find xxmaj felix and the two girls all crying is xxunk funny",". \n \n xxmaj for any fan of intelligent comedies , xxmaj the xxmaj odd xxmaj couple is a",". xxmaj xxmaj the the reason of the acting , xxmaj xxunk xxmaj xxunk xxmaj couple is a good xxunk"
"back to the xxup uk , i xxunk to my friends "" xxmaj xxunk for the worst movie ever made","! "" xxmaj oh , cruel xxmaj xxunk . xxmaj there is absolutely xxup nothing to recommend this film .",". "" xxmaj the , i , xxunk , xxmaj the 's no no nothing to do . film ."
xxmaj xxunk is so so cheesy that xxmaj xxunk have left xxmaj xxunk and xxmaj xxunk behind . \n \n,xxmaj there was not a single moment during the movie where i can say ' xxmaj oh ! at,"xxunk are a a xxunk xxunk of the film , xxmaj was see that xxmaj xxunk , xxmaj the i"
"xxunk , trust , and loyalty . xxmaj in more xxunk times , the fresh natural beauty of xxmaj xxunk","xxmaj harris , who plays xxmaj xxunk , would have had the men throwing the xxunk and xxunk at her","'s xxunk 's the is the xxunk , is be been to xxunk xxunk her xxunk xxunk xxunk xxunk her"


### Classification

In [36]:
data['class'] = (TextList.from_csv(path['imdb'], 'texts.csv',  cols='text', vocab = data['lm'].vocab)
                 .split_from_df(col='is_valid')
                 .label_from_df(cols='label')
                 .databunch(bs=32)
                )

In [38]:
data['class'].show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the xxunk - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the xxunk is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj xxunk",positive
"xxbos xxmaj many xxunk that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk xxunk experience in general . xxmaj with graphics",positive


In [39]:
learn = text_classifier_learner(data['class'], AWD_LSTM)
learn.load_encoder('imdb-qs-encoder-1')
learn.fit_one_cycle(2, slice(1e-3, 1e-2))
learn.save('classify-imdb-qs-1')

epoch,train_loss,valid_loss,accuracy,time
0,0.662780,0.601386,0.720000,00:03
1,0.620511,0.529115,0.760000,00:03


In [40]:
learn.show_results()

text,target,prediction
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive,positive
"xxbos xxmaj to review this movie , i without any doubt would have to xxunk that memorable scene in xxmaj tarantino 's "" xxmaj pulp xxmaj fiction "" ( xxunk ) when xxmaj xxunk and xxmaj xxunk are talking about xxmaj xxunk xxmaj wallace and what she does for a living . xxmaj xxunk tells xxmaj xxunk that the "" xxmaj only thing she did worthwhile was pilot "" .",negative,negative
"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one",negative,negative
"xxbos xxmaj the trouble with the book , "" xxmaj memoirs of a xxmaj geisha "" is that it had xxmaj japanese xxunk but underneath the xxunk it was all an xxmaj american man 's way of thinking . xxmaj reading the book is like watching a magnificent ballet with great music , sets , and costumes yet performed by xxunk animals dressed in those xxunk far from xxmaj japanese",negative,negative
"xxbos xxmaj bonanza had a great cast of wonderful actors . xxmaj xxunk xxmaj xxunk , xxmaj pernell xxmaj whitaker , xxmaj michael xxmaj xxunk , xxmaj dan xxmaj blocker , and even xxmaj guy xxmaj williams ( as the cousin who was brought in for several episodes during 1964 to replace xxmaj adam when he was leaving the series ) . xxmaj the cast had chemistry , and they",positive,positive


In [43]:
import json, fire, re
!python notebook2script.py langmodel-classify-imdb-qs.ipynb

Converted langmodel-classify-imdb-qs.ipynb to exp/nb_langmodel-classify-imdb-qs.py
